In [1]:
import fitsio
import numpy as np
import fitstools
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.colors import LinearSegmentedColormap
import balltracking.mballtrack as mblt
from skimage.exposure import rescale_intensity
import glob
from pathlib import PurePath
DTYPE = np.float32

In [2]:
def custom_cmap(nballs):
    #nballs = mbt_p.nballs + mbt_n.nballs
    # ## Custom colors => This must add a unique color for the background
    #colors = plt.cm.Set1_r(np.linspace(0, 1, 9))
    colors = plt.cm.Set3(np.linspace(0, 1, 9))
    # colors = plt.cm.Dark2(np.linspace(0, 1, 8))
    # # light gray color
    gray = np.array([[220, 220, 220, 255]]) / 255

    cmap = matplotlib.colors.ListedColormap(colors, name='mycmap', N=nballs)
    colors2 = np.array([cmap(i) for i in range(nballs)])
    # Add unique background color
    colors2 = np.concatenate((gray, colors2), axis=0)
    cmap2 = matplotlib.colors.ListedColormap(colors2, name='mycmap2')

    return cmap2


def plot_mballtrack(args, datafiles, outputdir):
    pos_p, pos_n, watershed_labels, borders, i = args

    data = fitstools.fitsread(datafiles, tslice=i).astype(DTYPE)


    plt.figure(figsize=(18, 8))

    plt.subplot(131)
    plt.imshow(data, cmap='gray', vmin=-100, vmax=100)
    #plt.colorbar()
    maskp = pos_p[0, :] > 0
    maskn = pos_n[0, :] > 0
    plt.scatter(pos_p[0, maskp], pos_p[1, maskp],marker='.', s=2, color='red')
    plt.scatter(pos_n[0, maskn], pos_n[1, maskn],marker='.', s=2, color='cyan')

    plt.xlabel('Lambert cyl. X')
    plt.ylabel('Lambert cyl. Y')


    plt.subplot(132)
    #bkg_gray = np.full([*data.shape, 3], 220, dtype=np.uint8)
    bkg_gray = rescale_intensity(np.tile(data[..., np.newaxis], (1,1,3)), in_range=(-100,100), out_range=np.uint8).astype(np.uint8)
    borders_rgb = bkg_gray.copy()
    # Color positive borders as red
    borders_rgb[borders == 1, 0] = 255
    borders_rgb[borders == 1, 1] = 0
    borders_rgb[borders == 1, 2] = 0
    # Color negative borders as cyan (accounting for color blindness)
    borders_rgb[borders == -1, 0] = 175
    borders_rgb[borders == -1, 1] = 238
    borders_rgb[borders == -1, 2] = 238

    #borders_rgb = np.concatenate((borders_red, borders_green, bkg_blue), axis=2)
    #plt.imshow(data, cmap='gray', vmin=-100, vmax=100)
    #plt.imshow(borders, vmin=0, vmax=1, origin='lower', cmap='Blues')
    plt.imshow(borders_rgb, origin='lower')
    plt.scatter(pos_p[0, maskp], pos_p[1, maskp], marker='.', s=2, color='red')
    # plt.text(x + 1, y + 1, '%d' % target, fontsize=8)
    plt.scatter(pos_n[0, maskn], pos_n[1, maskn], marker='.', s=2, color='cyan')
    plt.title('Boundaries of tracked fragments')


    plt.subplot(133)
    cmap = custom_cmap()
    plt.imshow(watershed_labels, cmap=cmap, vmin=-1, vmax=mbt_p.nballs + mbt_n.nballs, interpolation='nearest', origin='lower')
    plt.title('Tracked fragments with ball-based color labeling)')
    plt.tight_layout()

#     plt.savefig(PurePath(outputdir, 'frame_%02d.png'.format(i)))
#     plt.close()


In [3]:
def draw_borders(data, borders_p, borders_n):
    
    borders = borders_p.copy().astype(np.int8)
    borders[borders_n] = -1
    
    bkg_gray = rescale_intensity(np.tile(data[..., np.newaxis], (1,1,3)), in_range=(-100,100), out_range=np.uint8).astype(np.uint8)
    borders_rgb = bkg_gray.copy()
    # Color positive borders as red
    borders_rgb[borders == 1, 0] = 255
    borders_rgb[borders == 1, 1] = 0
    borders_rgb[borders == 1, 2] = 0
    # Color negative borders as cyan (accounting for color blindness)
    borders_rgb[borders == -1, 0] = 175
    borders_rgb[borders == -1, 1] = 238
    borders_rgb[borders == -1, 2] = 238
    
    return borders_rgb, borders


In [6]:
%%time
datadir = PurePath('/Users/rattie/Data/SDO/HMI/Peter/dataset1/magnetograms')
datafiles = sorted(glob.glob(str(PurePath(datadir, '*.fits'))))

mbt_dict = {"nt":100,
            "rs":2,
            "am":0.5,
            "dp":0.3,
            "td":1,
            "ballspacing":5,
            "intsteps":3,
            "mag_thresh":50,
            "noise_level":15,
            "track_emergence":True,
            "datafiles":datafiles,
            "do_plots":False,
            "fig_dir":PurePath(datadir, 'figures', 'debug')}

mbt_p, mbt_n = mblt.mballtrack_main(**mbt_dict)
# Convert to list of positions
list_pos_p = [mbt_p.ballpos[0:2, :, i] for i in range(mbt_dict['nt'])]
list_pos_n = [mbt_n.ballpos[0:2, :, i] for i in range(mbt_dict['nt'])]
#
# # Flux extraction by markers-based watershed
ws_list_p, markers_list_p, borders_list_p = mblt.watershed_series(mbt_dict['datafiles'], mbt_dict['nt'], mbt_dict['noise_level'], 1, mbt_p.ballpos.astype(np.int32))
ws_list_n, markers_list_n, borders_list_n = mblt.watershed_series(mbt_dict['datafiles'], mbt_dict['nt'], mbt_dict['noise_level'], -1, mbt_n.ballpos.astype(np.int32))



CPU times: user 8.32 s, sys: 513 ms, total: 8.83 s
Wall time: 9.3 s


In [ ]:
len(datafiles)

In [ ]:
mag = fitsio.read(datafiles[0])
mag = np.abs(mag)
np.std(mag)

In [ ]:
mbt_p.nballs, ws_list_p[7].max(), ws_list_p[8].max()

In [ ]:
cmap = custom_cmap(mbt_p.nballs+1)
cmap2 = custom_cmap(mbt_n.nballs+1)
bounds = np.arange(mbt_p.nballs)
norm = colors.BoundaryNorm(bounds, mbt_p.nballs)


fig,ax = plt.subplots(1, 2, figsize=(20,10))
ax[0].imshow(ws_list_p[0]+1, origin='lower', cmap=cmap, vmin=0, vmax=mbt_p.nballs)
ax[1].imshow(ws_list_n[0]+1, origin='lower', cmap=cmap2, vmin=0, vmax=mbt_n.nballs)

In [ ]:
cmap = custom_cmap(mbt_p.nballs + 1)


for i in range(mbt_dict['nt']):
    data = ws_list_p[i]+1
    data[data > 1] = 1
    fig,ax = plt.subplots(1, 2, figsize=(20,10))
    ax[0].imshow(data, origin='lower', cmap='gray')
    ax[0].plot(list_pos_p[i][0,:], list_pos_p[i][1,:], 'r+')
    ax[0].set_xlabel('X [px]')
    ax[0].set_ylabel('Y [px]')
    ax[0].axis([0, 512, 0, 512])
    for j in range(mbt_p.nballs):
        ax[0].text(list_pos_p[i][0,j]+5, list_pos_p[i][1,j], str(j), color='cyan')

    ax[1].imshow(ws_list_p[i]+1, origin='lower', cmap=cmap, vmin=0, vmax=mbt_p.nballs)
    ax[1].set_xlabel('X [px]')
    ax[1].set_ylabel('Y [px]')
    plt.savefig(PurePath(datadir, 'figures', 'fig_{:03d}.png'.format(i)))
    plt.close()

In [ ]:
cmap = custom_cmap(mbt_p.nballs + 1)


i = 0 
mag = fitsio.read(datafiles[i])
mag_borders, borders = draw_borders(mag, borders_list_p[i], borders_list_n[i])

bkg_gray = rescale_intensity(np.tile(mag[..., np.newaxis], (1,1,3)), in_range=(-100,100), out_range=np.uint8).astype(np.uint8)

fig, ax = plt.subplots(1, 2, figsize=(20, 10))
ax[0].imshow(mag_borders, origin='lower')

ax[1].imshow(bkg_gray, origin='lower')
# ax[1].contour(borders, levels=[1, 1], colors='cyan', linewidths=1, antialiased=True)
#ax[1].contour(ws_list_p[i]>0, levels=[1, 1], colors='red', linewidths=1, antialiased=True)
ax[1].contour(ws_list_n[i]+1, levels=np.arange(mbt_p.nballs)+1, cmap=cmap, linewidths=1)


In [ ]:
i = 0 
mag = fitsio.read(datafiles[i])
pos_p = mbt_p.ballpos[0:2, :, i]
pos_n = mbt_n.ballpos[0:2, :, i]

fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.imshow(mag, origin='lower', cmap='gray', vmin=-100, vmax=100)

maskp = pos_p[0, :] > 0
maskn = pos_n[0, :] > 0
plt.scatter(pos_p[0, maskp], pos_p[1, maskp],marker='o', s=6, color='red')
plt.scatter(pos_n[0, maskn], pos_n[1, maskn],marker='o', s=6, color='cyan')

In [ ]:
for i in range(mbt_dict['nt']):
    mag = fitsio.read(datafiles[i])
    pos_p = mbt_p.ballpos[0:2, :, i]
    pos_n = mbt_n.ballpos[0:2, :, i]

    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    ax.imshow(mag, origin='lower', cmap='gray', vmin=-100, vmax=100)
    ax.set_xlabel('CEA X [px]')
    ax.set_ylabel('CEA Y [px]')

    maskp = pos_p[0, :] > 0
    maskn = pos_n[0, :] > 0
    plt.scatter(pos_p[0, maskp], pos_p[1, maskp],marker='o', s=6, color='red')
    plt.scatter(pos_n[0, maskn], pos_n[1, maskn],marker='o', s=6, color='cyan')
    plt.savefig(PurePath(datadir, 'figures', 'fig_1panel_{:03d}.png'.format(i)))
    plt.close()

In [ ]:
mbt_p['nt']

In [ ]:
from bokeh.plotting import figure, output_file, show

In [ ]:
d = ws_list_p[7]

p = figure(tooltips=[("x", "$x"), ("y", "$y"), ("value", "@image")])
p.x_range.range_padding = p.y_range.range_padding = 0

# must give a vector of image data for image parameter
p.image(image=[d], x=0, y=0, dw=10, dh=10, palette="Spectral11", level="image")

show(p)

In [ ]:
d = ws_list_n[0]

p = figure(tooltips=[("x", "$x"), ("y", "$y"), ("value", "@image")])
p.x_range.range_padding = p.y_range.range_padding = 0

# must give a vector of image data for image parameter
p.image(image=[d], x=0, y=0, dw=10, dh=10, palette="Spectral11", level="image")

show(p)